In [52]:
import pandas as pd
from pprint import pprint #print nested map in better format way
from IPython.display import Image

In [53]:
! pip install -q kaggle

In [54]:
from google.colab import files

In [55]:
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"srvsakshi03","key":"41190edae2633d15326ff3cf72335b85"}'}

In [56]:
! mkdir ~/.kaggle


mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [57]:
! cp kaggle.json ~/.kaggle/

In [58]:
! chmod 600 ~/.kaggle/kaggle.json

In [59]:
#below method helped me to download datasets that are not listed in the kaggle competitions.
!kaggle datasets download -d gaveshjain/ford-sentence-classifiaction-dataset -p /content/drive/MyDrive/ford-sentence-classification-dataset/ --unzip


 69% 2.00M/2.92M [00:00<00:00, 2.43MB/s]
100% 2.92M/2.92M [00:01<00:00, 2.88MB/s]


In [60]:
pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [61]:
import splitfolders

In [62]:
traindf=pd.read_csv("/content/drive/MyDrive/ford-sentence-classification-dataset/train_data.csv" ,index_col="Sentence_id",keep_default_na=False)
testdf=pd.read_csv("/content/drive/MyDrive/ford-sentence-classification-dataset/test_data.csv",index_col="Sentence_id",keep_default_na=False)
submission_labels=pd.read_csv("/content/drive/MyDrive/ford-sentence-classification-dataset/sample_submission.csv",squeeze=True)
del traindf['Unnamed: 0']


In [63]:
traindf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60115 entries, GERRES15609 to HONEDU1643
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   New_Sentence  60115 non-null  object
 1   Type          60115 non-null  object
dtypes: object(2)
memory usage: 1.4+ MB


In [64]:
testdf.head()

,Unnamed: 0,New_Sentence
Sentence_id,,
UAERES16346,0,Collaborate across all of DB&T practices and o...
COGREQ15586,1,Strong technology expertise in Identity and A...
UAEREQ12722,2,Strong knowledge on Service Virtualization.
COGSKL29155,3,Architect scalable data processing and analyti...
PHERES12551,4,"Map client organization, build outstanding rel..."


In [65]:
traindf['Type'].value_counts()

Responsibility    15561
Requirement       14132
SoftSkill          9581
Experience         9248
Skill              6956
Education          4637
Name: Type, dtype: int64

**UnderSampling**

In [66]:
#under sample
def undersample(df):
  min_value_count=min(df['Type'].value_counts())
  # print(min_value_count)
  undersampledf = pd.DataFrame()
  length = df['Type'].value_counts()
  skillDF = df[df['Type'] == "Skill"].sample(min_value_count)
  RequirementDF = df[df['Type'] == "Requirement"].sample(min_value_count)
  ResponsibilityDF = df[df['Type'] == "Responsibility"].sample(min_value_count)
  ExperienceDF = df[df['Type'] == "Experience"].sample(min_value_count)
  SoftSkillDF = df[df['Type'] == "SoftSkill"].sample(min_value_count)
  EducationDF = df[df['Type'] == "Education"].sample(min_value_count)
  undersampledf=skillDF
  undersampledf=undersampledf.append(ResponsibilityDF)
  undersampledf=undersampledf.append(RequirementDF)
  undersampledf=undersampledf.append(ExperienceDF)
  undersampledf=undersampledf.append(SoftSkillDF)
  undersampledf=undersampledf.append(EducationDF)
  
  typelist=[]
  typelist.append(df['Type'].unique())
 
  # for i in range(0,len(typelist)):
  #   print(typelist[i],i)
  #   #x=x.append(df[df['Type'] == df['Type'][i]].sample(min_value_count))
  #   x=x.append(df[df['Type'] == "Skill"].sample(min_value_count))
  print("After loop:")
    
  print(undersampledf['Type'].value_counts())
  return undersampledf

In [67]:
undersample_df=undersample(traindf)

After loop:
Skill             4637
Responsibility    4637
Requirement       4637
Experience        4637
SoftSkill         4637
Education         4637
Name: Type, dtype: int64


**Over Sampling**

In [68]:
def oversample(df):
  max_value_count=max(df['Type'].value_counts())
  # print(min_value_count)
  oversampledf = pd.DataFrame()
  length = df['Type'].value_counts()
  skillDF = df[df['Type'] == "Skill"].sample(max_value_count,replace=True)
  RequirementDF = df[df['Type'] == "Requirement"].sample(max_value_count,replace=True)
  ResponsibilityDF = df[df['Type'] == "Responsibility"].sample(max_value_count,replace=True)
  ExperienceDF = df[df['Type'] == "Experience"].sample(max_value_count,replace=True)
  SoftSkillDF = df[df['Type'] == "SoftSkill"].sample(max_value_count,replace=True)
  EducationDF = df[df['Type'] == "Education"].sample(max_value_count,replace=True)
  oversampledf=skillDF
  oversampledf=oversampledf.append(ResponsibilityDF)
  oversampledf=oversampledf.append(RequirementDF)
  oversampledf=oversampledf.append(ExperienceDF)
  oversampledf=oversampledf.append(SoftSkillDF)
  oversampledf=oversampledf.append(EducationDF)
  print("After loop:")
    
  print(oversampledf['Type'].value_counts())
  return oversampledf


In [69]:
oversample_df=oversample(traindf)

After loop:
Skill             15561
Responsibility    15561
Requirement       15561
Experience        15561
SoftSkill         15561
Education         15561
Name: Type, dtype: int64


In [70]:
traindf['Type'].value_counts()

Responsibility    15561
Requirement       14132
SoftSkill          9581
Experience         9248
Skill              6956
Education          4637
Name: Type, dtype: int64

In [71]:
#import regular expression
import re
#data preprocessing by regualar expression 
def dataPreprocessing(text):
    text=re.sub(r'[^\w\s\t\']',' ',text)
    text=re.sub(r' +',' ',text)
    return text.strip().lower()

In [72]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [73]:

stops = set(stopwords.words('english'))
def datacleaning(New_Sentence):
  print("New_Sentence",New_Sentence)
  remove_punctuation=[char for char in New_Sentence if char not in string.punctuation]
  remove_punctuation=''.join(remove_punctuation)
  print("remove_punctuation",remove_punctuation)
  return [word for word in remove_punctuation.split() if word.lower() not in stopwords.words('english')]
    


In [74]:
def get_touple():
  touples=[]#(type,text)
  for i in range(0,len(traindf)):
    value=(traindf['Type'][i],traindf['New_Sentence'][i])
    touples.append(value)
  return touples

In [75]:
from nltk.tokenize.punkt import FreqDist
import nltk
from nltk import word_tokenize
nltk.download('punkt')
from collections import defaultdict
import collections
import string


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [76]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [77]:
def datacleaning(New_Sentence):
  remove_punctuation=[char for char in New_Sentence if char not in string.punctuation]
  remove_punctuation=''.join(remove_punctuation)
  return [word for word in remove_punctuation.split() if word.lower() not in stopwords.words('english')]


In [78]:
#whole line splitted into words
def get_frequency(touples):
  
  map=collections.defaultdict(list)
  for touple in touples:
    label=touple[0]
    text=touple[1]
    wordcounts=datacleaning(text)
    map[label].extend(wordcounts)
  
  for c_label,c_tokens in map.items():
    print("\u0332".join("\n label:"),c_label)
    frequency=FreqDist(c_tokens)
    print(frequency.most_common(20))
  return map
  

In [79]:
touples=get_touple()
#give running word counts of each label(how many times each word occurs in all label/typelist)

In [80]:
map=get_frequency(touples)



̲ ̲l̲a̲b̲e̲l̲: Responsibility
[('business', 1595), ('team', 1430), ('customer', 1116), ('support', 1032), ('development', 956), ('project', 932), ('management', 838), ('technical', 837), ('product', 777), ('requirements', 756), ('design', 754), ('process', 749), ('customers', 715), ('Honeywell', 685), ('work', 672), ('projects', 669), ('ensure', 665), ('teams', 651), ('new', 632), ('data', 620)]

̲ ̲l̲a̲b̲e̲l̲: Requirement
[('experience', 2153), ('Experience', 1779), ('Ability', 1442), ('work', 1082), ('knowledge', 1067), ('skills', 1050), ('ability', 973), ('management', 709), ('business', 659), ('working', 649), ('Must', 638), ('required', 637), ('degree', 566), ('Strong', 558), ('development', 555), ('environment', 519), ('Knowledge', 499), ('understanding', 468), ('able', 465), ('team', 460)]

̲ ̲l̲a̲b̲e̲l̲: Skill
[('experience', 1155), ('Experience', 1131), ('knowledge', 640), ('Knowledge', 441), ('management', 344), ('tools', 329), ('etc', 300), ('skills', 298), ('understanding'

In [81]:
from scipy.stats import multinomial
from nltk.tag.sequential import NaiveBayesClassifier
from numpy.lib.function_base import vectorize
import random

In [82]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import pickle
from sklearn.metrics import *
import sklearn.metrics as metrics

In [83]:
def splits(touples):
  random.shuffle(touples)
  X_train,Y_train,X_val,Y_val=[],[],[],[]
  intersection=int(0.80*len(touples)) #convert into into
  for i in range(0,intersection):
    X_train.append(touples[i][1])
    Y_train.append(touples[i][0])

  for i in range(intersection,len(touples)):
    X_val.append(touples[i][1])
    Y_val.append(touples[i][0])
  
  return X_train,Y_train,X_val,Y_val

min_df is used for removing terms that appear too infrequently. For example: min_df = 0.01 means "ignore terms that appear in less than 1% of the documents". Refernce:

https://stackoverflow.com/questions/27697766/understanding-min-df-and-max-df-in-scikit-countvectorizer

In [84]:
from nltk.corpus.reader import ycoe
def evaluate_classifier(title,Classifier,transformer,X_test,y_test):
  
  Xtesttfidf=transformer.transform(X_test)
  y_pred=Classifier.predict(Xtesttfidf)
  precision=metrics.precision_score(y_test,y_pred,average='micro')

  
  print("precision-",precision)
  recall=metrics.recall_score(y_test,y_pred,average='micro')
  print("recall",recall)
  f1=metrics.f1_score(y_test,y_pred,average='micro')
  print("f1",f1)
  print("%s\t%f\t%f\t%f"%(title,precision,recall,f1))

def evaluate(title, y, predictions):
  precision = metrics.precision_score(y, predictions, average='micro')  
  print("precision-",precision)
  recall=metrics.recall_score(y,predictions,average='micro')
  print("recall",recall)
  f1=metrics.f1_score(y,predictions,average='micro')
  print("f1",f1)
  print("%s\t%f\t%f\t%f"%(title,precision,recall,f1))



**SMOTE(oversampling by producing synthetic samples-uses K-Nearest Neighbors Algorithm)**

In [85]:
from imblearn.pipeline import Pipeline, make_pipeline
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfTransformer

def create_pipeline(sampler_type):
  if sampler_type=='smote':
    return Pipeline([
      ('vect', CountVectorizer(stop_words='english',min_df=1,analyzer='word',ngram_range=(1,1))),
      ('tfidf', TfidfTransformer()),
      ('smote', SMOTE(random_state=12)),
      ('mnb', MultinomialNB(alpha=0.1))
    ]) 
  elif sampler_type=='near_miss':
    return Pipeline([
      ('vect', CountVectorizer(stop_words='english',min_df=1,analyzer='word',ngram_range=(1,1))),
      ('tfidf', TfidfTransformer()),
      ('nm', NearMiss()),
      ('mnb', MultinomialNB(alpha=0.1))
  ])
  elif sampler_type=='EditedNearestNeighbours':
    return Pipeline([
      ('vect', CountVectorizer(stop_words='english',min_df=1,analyzer='word',ngram_range=(1,1))),
      ('tfidf', TfidfTransformer()),
      ('enn', EditedNearestNeighbours()),
      ('mnb', MultinomialNB(alpha=0.1))
  ])
  elif sampler_type=='RepeatedEditedNearestNeighbours':
    return Pipeline([
      ('vect', CountVectorizer(stop_words='english',min_df=1,analyzer='word',ngram_range=(1,1))),
      ('tfidf', TfidfTransformer()),
      ('renn', RepeatedEditedNearestNeighbours()),
      ('mnb', MultinomialNB(alpha=0.1))
  ])
  else:
    return Pipeline([
      ('vect', CountVectorizer(stop_words='english',min_df=1,analyzer='word',ngram_range=(1,1))),
      ('tfidf', TfidfTransformer()),
      ('mnb', MultinomialNB(alpha=0.1))
  ])

**Naive Bayes Implementation**

In [86]:
import math
import operator
from collections import Counter

class FordSentenceClassifier():
    def clean(self, s):
        translator = str.maketrans("", "", string.punctuation)
        return s.translate(translator)

    def tokenize(self, text):
        text = self.clean(text).lower()
        return re.split("\W+", text)

    def get_word_counts(self, words):
        word_counts = {}
        for word in words:
            word_counts[word] = word_counts.get(word, 0.0) + 1.0
        return word_counts

    # Probability of occurence
    # P[“the”] = num of documents containing ‘the’ / num of all documents
    def p(self, word, X):
      n = len(X)
      count = 0
      for label in self.labels:
        count += self.word_counts[label][word]
      return count/n

    # Conditional probability based on the sentiment
    # P[“the” | Requirement]  = # of Requirement sentences containing “the” / num of all Requirement sentences
    def conditional_p(self, word):
      conditional_p = {}
      for label in self.labels:        
        n = self.word_counts[label][word]
        total = self.num_messages[label]
        conditional_p[label] = n/total
      return conditional_p

    # Top 10 words that predict each class
    def top(self, n):
      top_words = {}
      for label in self.labels:
        top_words[label] = dict(Counter(self.word_counts[label]).most_common(5))
      return top_words


    def fit(self, X, Y):
      self.num_messages = {}
      self.log_class_priors = {}
      self.word_counts = {}
      self.vocab = set()
      n = len(X)
      self.labels = set(Y)

      for label in self.labels:
        self.num_messages[label] = sum(1 for item in Y if item == label)
        #print(f"Number of {label}", self.num_messages[label])
        self.log_class_priors[label] = math.log(self.num_messages[label] / n)
        self.word_counts[label] = {}
      
      for x, y in zip(X, Y):
          c = y
          counts = self.get_word_counts(self.tokenize(x))
          for word, count in counts.items():
              if word not in self.vocab:
                  self.vocab.add(word)
              if word not in self.word_counts[c]:
                  self.word_counts[c][word] = 0.0
              self.word_counts[c][word] += count

    def predict(self, X, smoothing=False):
      result = []
      for x in X:
          counts = self.get_word_counts(self.tokenize(x))
          score = {}
          for word, _ in counts.items():
              if word not in self.vocab:
                continue
              
              # https://en.wikipedia.org/wiki/Laplacian_smoothing
              for label in self.labels:
                log_w_given = math.log( (self.word_counts[label].get(word, 0.0) + 1 if smoothing else 0) / (self.num_messages[label] + len(self.vocab) if smoothing else 0))
                if label not in score:
                  score[label] = 0
                score[label] += log_w_given
              
          for label in self.labels:
            if label not in score:
              score[label] = 0
            score[label] += self.log_class_priors[label]       
          result.append(max(score.items(), key=operator.itemgetter(1))[0])            
      return result

In [111]:
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import NearMiss
from imblearn.under_sampling import EditedNearestNeighbours 
from imblearn.under_sampling import RepeatedEditedNearestNeighbours
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter

def testing(testdf, textclassifier):
  testsentences=[]
  for i in range(0,len(testdf)):
    value=(testdf['New_Sentence'][i])
    testsentences.append(value)
  
  predictions=textclassifier.predict(testsentences)
  return predictions
  

def training_with_us(touples, us_type):  
  y = list(zip(*touples))[0]
  x = list(zip(*touples))[1]
  
  x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y,random_state=0)
  
  textclassifier = create_pipeline(us_type).fit(x_train,y_train)
  predictions=textclassifier.predict(x_train)
  
  print("\u0332".join(f"\n {us_type} technique \n"))
  print("\u0332".join("\n With Train \n"))
  evaluate("NaiveBayes\tTRAIN\t", y_train, predictions)

  test_pred=textclassifier.predict(x_test)
  print("\u0332".join("\n with Test \n"))
  evaluate("NaiveBayes\tTEST\t", y_test, test_pred)
  return textclassifier

def training_with_custom_nb(touples, smoothing=False, n_split=0.75):
  y = list(zip(*touples))[0]
  X = list(zip(*touples))[1]  
  num_train = int(n_split*len(X))

  print("\u0332".join("\n With Train \n"))
  MNB = FordSentenceClassifier()  
  MNB.fit(X[:num_train], y[:num_train])
  pred = MNB.predict(X[:num_train], smoothing)
  true = y[:num_train]
  accuracy = sum(1 for i in range(len(pred)) if pred[i] == true[i]) / float(len(pred))
  print("Accuracy: {0:.4f}".format(accuracy))  
  
  print("\u0332".join("\n With Test \n"))  
  pred = MNB.predict(X[num_train+1:], smoothing)
  true = y[num_train+1:]
  accuracy = sum(1 for i in range(len(pred)) if pred[i] == true[i]) / float(len(pred))
  print("Accuracy: {0:.4f}".format(accuracy))  

  print("\u0332".join("\n Probabilities \n"))  
  print("Probability for word (the) is:", MNB.p("the", X))
  print("\n")
  top_words = MNB.top(10)
 
  print("\u0332".join("top 10 words for each class"))
  print("\n")
  for k in top_words:
    print(k, top_words[k])
  print("\u0332".join("Conditional probabilty for word (the) given label"))
  cond_prob=MNB.conditional_p("the")
  print("\n")
  for k in cond_prob:
    print(k, cond_prob[k])
  return MNB

def test_with_custom_nb(testdf, model):
  testsentences=[]
  for i in range(0,len(testdf)):
    value=(testdf['New_Sentence'][i])
    testsentences.append(value)
  print("\u0332".join("\n Predictions \n")) 
  predictions=model.predict(testsentences[:10],smoothing=True)
  return predictions


In [112]:
#textclassifier=training_with_us(touples,'RepeatedEditedNearestNeighbours')
mnb = training_with_custom_nb(touples, smoothing=True)
predictions = test_with_custom_nb(testdf, mnb)
print(predictions[:5])


̲ ̲W̲i̲t̲h̲ ̲T̲r̲a̲i̲n̲ ̲

Accuracy: 0.6910

̲ ̲W̲i̲t̲h̲ ̲T̲e̲s̲t̲ ̲

Accuracy: 0.6528

̲ ̲P̲r̲o̲b̲a̲b̲i̲l̲i̲t̲i̲e̲s̲ ̲

Probability for word (the) is: 0.16686351160276136


t̲o̲p̲ ̲1̲0̲ ̲w̲o̲r̲d̲s̲ ̲f̲o̲r̲ ̲e̲a̲c̲h̲ ̲c̲l̲a̲s̲s


Experience {'experience': 6345.0, 'years': 5962.0, 'of': 5348.0, 'in': 3856.0, 'and': 2677.0}
Requirement {'and': 7196.0, 'to': 4675.0, 'in': 3432.0, 'experience': 2952.0, 'of': 2931.0}
Responsibility {'and': 12570.0, 'the': 5912.0, 'to': 5268.0, 'of': 4165.0, 'with': 2729.0}
Education {'or': 2115.0, 'in': 2027.0, 'degree': 1890.0, 'engineering': 1430.0, 'bachelors': 1022.0}
Skill {'and': 2420.0, 'experience': 1710.0, 'in': 1381.0, 'of': 1357.0, 'with': 917.0}
SoftSkill {'and': 5164.0, 'to': 2918.0, 'skills': 2423.0, 'ability': 1543.0, 'in': 1279.0}
C̲o̲n̲d̲i̲t̲i̲o̲n̲a̲l̲ ̲p̲r̲o̲b̲a̲b̲i̲l̲t̲y̲ ̲f̲o̲r̲ ̲w̲o̲r̲d̲ ̲(̲t̲h̲e̲)̲ ̲g̲i̲v̲e̲n̲ ̲l̲a̲b̲e̲l


Experience 0.10328366845074498
Requirement 0.18052278946871755
Responsibility 0.5065113091158328
Education 0.0744

In [89]:
textclassifier=training_with_us(touples,'RepeatedEditedNearestNeighbours')



̲ ̲R̲e̲p̲e̲a̲t̲e̲d̲E̲d̲i̲t̲e̲d̲N̲e̲a̲r̲e̲s̲t̲N̲e̲i̲g̲h̲b̲o̲u̲r̲s̲ ̲t̲e̲c̲h̲n̲i̲q̲u̲e̲ ̲


̲ ̲W̲i̲t̲h̲ ̲T̲r̲a̲i̲n̲ ̲

precision- 0.5562258794304219
recall 0.5562258794304219
f1 0.5562258794304219
NaiveBayes	TRAIN		0.556226	0.556226	0.556226

̲ ̲w̲i̲t̲h̲ ̲T̲e̲s̲t̲ ̲

precision- 0.5410872313527181
recall 0.5410872313527181
f1 0.5410872313527181
NaiveBayes	TEST		0.541087	0.541087	0.541087


In [90]:
textclassifier=training_with_us(touples,'EditedNearestNeighbours')


̲ ̲E̲d̲i̲t̲e̲d̲N̲e̲a̲r̲e̲s̲t̲N̲e̲i̲g̲h̲b̲o̲u̲r̲s̲ ̲t̲e̲c̲h̲n̲i̲q̲u̲e̲ ̲


̲ ̲W̲i̲t̲h̲ ̲T̲r̲a̲i̲n̲ ̲

precision- 0.5562258794304219
recall 0.5562258794304219
f1 0.5562258794304219
NaiveBayes	TRAIN		0.556226	0.556226	0.556226

̲ ̲w̲i̲t̲h̲ ̲T̲e̲s̲t̲ ̲

precision- 0.5410872313527181
recall 0.5410872313527181
f1 0.5410872313527181
NaiveBayes	TEST		0.541087	0.541087	0.541087


In [91]:
textclassifier=training_with_us(touples,'near_miss')


̲ ̲n̲e̲a̲r̲_̲m̲i̲s̲s̲ ̲t̲e̲c̲h̲n̲i̲q̲u̲e̲ ̲


̲ ̲W̲i̲t̲h̲ ̲T̲r̲a̲i̲n̲ ̲

precision- 0.6819855387481701
recall 0.6819855387481701
f1 0.6819855387481701
NaiveBayes	TRAIN		0.681986	0.681986	0.681986

̲ ̲w̲i̲t̲h̲ ̲T̲e̲s̲t̲ ̲

precision- 0.6357043050103134
recall 0.6357043050103134
f1 0.6357043050103134
NaiveBayes	TEST		0.635704	0.635704	0.635704


In [92]:
all_predictions=testing(testdf, textclassifier)

In [93]:
print(all_predictions)

['Responsibility' 'Skill' 'Skill' ... 'SoftSkill' 'Skill' 'Experience']


In [94]:
testdf['New_Sentence']

Sentence_id
UAERES16346    Collaborate across all of DB&T practices and o...
COGREQ15586    Strong  technology expertise in Identity and A...
UAEREQ12722         Strong knowledge on Service  Virtualization.
COGSKL29155    Architect scalable data processing and analyti...
PHERES12551    Map client organization, build outstanding rel...
                                     ...                        
HONEXP17084    A minimum of 3 years of experience in Industri...
UAESSK45553    Track record of managing multiple projects wit...
GERSSK14317    In addition, the candidate should have effecti...
PHEREQ4980     Candidate should have sufficient knowledge on ...
PHESKL2893     Experience in deployment and upgrading & imple...
Name: New_Sentence, Length: 15029, dtype: object

In [95]:
testdf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15029 entries, UAERES16346 to PHESKL2893
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    15029 non-null  int64 
 1   New_Sentence  15029 non-null  object
dtypes: int64(1), object(1)
memory usage: 868.3+ KB


In [96]:
testdf.head()

,Unnamed: 0,New_Sentence
Sentence_id,,
UAERES16346,0,Collaborate across all of DB&T practices and o...
COGREQ15586,1,Strong technology expertise in Identity and A...
UAEREQ12722,2,Strong knowledge on Service Virtualization.
COGSKL29155,3,Architect scalable data processing and analyti...
PHERES12551,4,"Map client organization, build outstanding rel..."


In [97]:
newtestdf=testdf['New_Sentence'].to_frame()

# [newtestdf.columns.tolist()]
items = newtestdf.reset_index().values.tolist()

print(type(items))
print(items)
sentence_ids = list(zip(*items))[0]
print(sentence_ids)

submission = pd.DataFrame({"Sentence_id": np.array(sentence_ids),"Type": all_predictions})
submission.to_csv('submission.csv', index=False)

<class 'list'>
[['UAERES16346', 'Collaborate across all of DB&T practices and offerings in AI, Data, IoT  and Cloud platforms to ensure we grow share in world class digital innovation, mobile/web applications helping clients reinvent their businesses.'], ['COGREQ15586', 'Strong  technology expertise in Identity and Access Management.'], ['UAEREQ12722', 'Strong knowledge on Service  Virtualization.'], ['COGSKL29155', 'Architect scalable data processing and analytics solutions, including technical feasibility for Big Data storage, processing, and consumption e.g., development  of enterprise Data Lake strategy, and heterogeneous data management.'], ['PHERES12551', 'Map client organization, build outstanding relationships with new business units,  and build a sales strategy for developing new business opportunities.'], ['INDRES5908', 'Assure compliance and recommend actions to improve the effectiveness of  the quality system.'], ['UAESKL39669', 'WE VALUEKnowledge of purchasing  system (Ora

In [98]:
submission = pd.read_csv('submission.csv')
submission.head(10)

,Sentence_id,Type
0,UAERES16346,Responsibility
1,COGREQ15586,Skill
2,UAEREQ12722,Skill
3,COGSKL29155,Experience
4,PHERES12551,Responsibility
5,INDRES5908,Responsibility
6,UAESKL39669,Skill
7,HONSSK10706,Responsibility
8,COGRES1719,Skill
9,HONREQ3632,Requirement
